<a href="https://bmi.readthedocs.io"><img src="./media/bmi-logo-header-text.png"></a>

# Run the `Heat` model through its BMI

`Heat` models the diffusion of temperature on a uniform rectangular plate with Dirichlet boundary conditions. This is the canonical example used in the [bmi-example-python](https://github.com/csdms/bmi-example-python) repository. View the source code for the [model](https://github.com/csdms/bmi-example-python/blob/master/heat/heat.py) and its [BMI](https://github.com/csdms/bmi-example-python/blob/master/heat/bmi_heat.py) on GitHub.

Start by importing `os`, `numpy` and the `Heat` BMI:

In [ ]:
import numpy as np
from heat import BmiHeat

Create an instance of the model's BMI.

In [ ]:
x = BmiHeat()

What's the name of this model?

In [ ]:
print(x.get_component_name())

Start the `Heat` model through its BMI using a configuration file:

In [ ]:
cat heat.yaml

In [ ]:
x.initialize("heat.yaml")

Check the time information for the model.

In [ ]:
print("Start time:", x.get_start_time())
print("End time:", x.get_end_time())
print("Current time:", x.get_current_time())
print("Time step:", x.get_time_step())
print("Time units:", x.get_time_units())

Show the input and output variables for the component (aside on [Standard Names](https://csdms.colorado.edu/wiki/CSDMS_Standard_Names)):

In [ ]:
print(x.get_input_var_names())
print(x.get_output_var_names())

Next, get the identifier for the grid on which the temperature variable is defined:

In [ ]:
grid_id = x.get_var_grid("plate_surface__temperature")
print("Grid id:", grid_id)

Then get the grid attributes:

In [ ]:
print("Grid type:", x.get_grid_type(grid_id))

rank = x.get_grid_rank(grid_id)
print("Grid rank:", rank)

shape = np.ndarray(rank, dtype=int)
x.get_grid_shape(grid_id, shape)
print("Grid shape:", shape)

spacing = np.ndarray(rank, dtype=float)
x.get_grid_spacing(grid_id, spacing)
print("Grid spacing:", spacing)

These commands are made somewhat un-Pythonic by the generic design of the BMI.

Through the model's BMI, zero out the initial temperature field, except for an impulse near the middle.
Note that *set_value* expects a one-dimensional array for input.

In [ ]:
temperature = np.zeros(shape)
temperature[3, 4] = 100.0
x.set_value("plate_surface__temperature", temperature)

Check that the temperature field has been updated. Note that *get_value* expects a one-dimensional array to receive output.

In [ ]:
temperature_flat = np.empty_like(temperature).flatten()
x.get_value("plate_surface__temperature", temperature_flat)
print(temperature_flat.reshape(shape))

Now advance the model by a single time step:

In [ ]:
x.update()

View the new state of the temperature field:

In [ ]:
x.get_value("plate_surface__temperature", temperature_flat)
print(temperature_flat.reshape(shape))

There's diffusion!

Advance the model to some distant time:

In [ ]:
distant_time = 2.0
while x.get_current_time() < distant_time:
    x.update()

View the final state of the temperature field:

In [ ]:
np.set_printoptions(formatter={"float": "{: 5.1f}".format})
x.get_value("plate_surface__temperature", temperature_flat)
print(temperature_flat.reshape(shape))

Note that temperature isn't conserved on the plate:

In [ ]:
print(temperature_flat.sum())

End the model:

In [ ]:
x.finalize()